# Aprendizaje reforzado

Este cuaderno de Jupyter actúa como material de apoyo para el **Capítulo 21 Aprendizaje por refuerzo** del libro* Inteligencia artificial: un enfoque moderno*. Este cuaderno utiliza las implementaciones del módulo `rl.py`. También utilizamos la implementación de MDP en el módulo `mdp.py` para probar nuestros agentes. Puede resultar útil si ya ha leído el cuaderno de Jupyter que trata sobre el proceso de decisión de Markov. Importemos todo desde el módulo `rl`. Podría resultar útil ver el código fuente de algunas de nuestras implementaciones. Consulte el cuaderno introductorio de Jupyter para obtener más detalles.

In [ ]:
from reinforcement_learning import *

## CONTENIDO

* Descripción general
* Aprendizaje pasivo por refuerzo
- Estimación de Utilidad Directa
- Programación dinámica adaptativa
- Agente de diferencia temporal
*Aprendizaje por refuerzo activo
- Q aprendizaje

## DESCRIPCIÓN GENERAL

Antes de comenzar a jugar con las implementaciones reales, revisemos un par de cosas sobre RL.

1. El aprendizaje por refuerzo se ocupa de cómo los agentes de software deberían tomar acciones en un entorno para maximizar alguna noción de recompensa acumulativa.

2. El aprendizaje por refuerzo se diferencia del aprendizaje supervisado estándar en que nunca se presentan los pares de entrada/salida correctos, ni se corrigen explícitamente las acciones subóptimas. Además, hay un enfoque en el desempeño en línea, que implica encontrar un equilibrio entre la exploración (de territorio inexplorado) y la explotación (del conocimiento actual).

-- Fuente: [Wikipedia](https://en.wikipedia.org/wiki/Reinforcement_learning)

En resumen, tenemos una secuencia de transiciones de acciones estatales con recompensas asociadas con algunos estados. Nuestro objetivo es encontrar la política óptima $\pi$ que nos diga qué acción tomar en cada estado.

## APRENDIZAJE PASIVO POR REFUERZO

En el aprendizaje por refuerzo pasivo, el agente sigue una política fija $\pi$. El aprendizaje pasivo intenta evaluar la política dada $pi$, sin ningún conocimiento de la función de recompensa $R(s)$ y el modelo de transición $P(s'\ |\ s, a)$.

Esto generalmente se hace mediante algún método de **estimación de utilidad**. El agente intenta aprender directamente la utilidad de cada estado que resultaría de seguir la política. Tenga en cuenta que en cada paso, tiene que *percibir* la recompensa y el estado; no tiene conocimiento global de estos. Por lo tanto, si un conjunto completo de acciones ofrece una probabilidad muy baja de alcanzar algún estado $s_+$, es posible que el agente nunca perciba la recompensa $R(s_+)$.

Considere una situación en la que a un agente se le da una política a seguir. Por lo tanto, en cualquier momento sólo conoce su estado actual y su recompensa actual, y la acción que debe realizar a continuación. Esta acción puede llevarlo a más de un estado, con diferentes probabilidades.

Para una serie de acciones dadas por $\pi$, la utilidad estimada $U$:
$$U^{\pi}(s) = E(\sum_{t=0}^\inf \gamma^t R^t(s')$$)
O el valor esperado de la suma de las recompensas descontadas hasta la terminación.

Con base en este concepto, analizamos tres métodos para estimar la utilidad:

1. **Estimación directa de utilidad (DUE)**

El primer método, el más ingenuo, de estimar la utilidad proviene de la interpretación más simple de la definición anterior. Construimos un agente que sigue la política hasta llegar al estado terminal. En cada paso, registra su estado actual, recompensa. Una vez que alcanza el estado terminal, puede estimar la utilidad de cada estado para *esa* iteración, simplemente sumando las recompensas descontadas de ese estado al estado terminal.

Ahora puede ejecutar esta 'simulación' $n$ veces y calcular la utilidad promedio de cada estado. Si un estado ocurre más de una vez en una simulación, ambos valores de utilidad se cuentan por separado.

Tenga en cuenta que este método puede resultar prohibitivamente lento para espacios de estados muy grandes. Además, **no presta atención a la probabilidad de transición $P(s'\ |\ s, a)$.** Se pierde información que es capaz de recopilar (digamos, registrando el número de veces que se realiza una acción). de un estado llevó a otro estado). El siguiente método aborda este problema.

2. **Programación dinámica adaptativa (ADP)**

Este método hace uso del conocimiento del estado pasado $s$, la acción $a$ y el nuevo estado percibido $s'$ para estimar la probabilidad de transición $P(s'\ |\ s,a)$. Lo hace mediante el simple recuento de nuevos estados resultantes de estados y acciones anteriores.<br>
El programa ejecuta la política varias veces, realizando un seguimiento de:
- cada aparición del estado $s$ y la acción recomendada por la política $a$ en $N_{sa}$
- cada aparición de $s'$ resultante de $a$ en $s$ en $N_{s'|sa}$.

Por lo tanto, puede estimar $P(s'\ |\ s,a)$ como $N_{s'|sa}/N_{sa}$, que en el límite de infinitas pruebas, convergerá al valor verdadero.<br >
Utilizando las probabilidades de transición así estimadas, se puede aplicar la "EVALUACIÓN DE POLÍTICAS" para estimar las utilidades $U(s)$ utilizando propiedades de convergencia de las funciones de Bellman.

3. **Aprendizaje de diferencia temporal (TD)**

En lugar de construir explícitamente el modelo de transición $P$, el modelo de diferencia temporal hace uso de la cercanía esperada entre las utilidades de dos estados consecutivos $s$ y $s'$.
Para la transición $s$ a $s'$, la actualización se escribe como:
$$U^{\pi}(s) \leftarrow U^{\pi}(s) + \alpha \left( R(s) + \gamma U^{\pi}(s') - U^{\ pi}(s) \derecha)$$
Este modelo incorpora implícitamente las probabilidades de transición al sopesar cada estado por el número de veces que se alcanza desde el estado actual. Por tanto, tras varias iteraciones, converge de manera similar a las ecuaciones de Bellman.
La ventaja del modelo de aprendizaje TD es su cálculo relativamente simple en cada paso, en lugar de tener que realizar un seguimiento de varios conteos.
Para $n_s$ estados y $n_a$ acciones, el modelo ADP tendría $n_s \times n_a$ números $N_{sa}$ y $n_s^2 \times n_a$ números $N_{s'|sa}$ para realizar un seguimiento de. El modelo TD solo debe realizar un seguimiento de una utilidad $U(s)$ para cada estado.

#### Demostrando agentes pasivos

Los agentes pasivos se implementan en `rl.py` como varias `Clase de agente`.

Para demostrar estos agentes, utilizamos el objeto `GridMDP` del módulo `MDP`. `sequential_decision_environment` es similar al usado para el portátil `MDP` pero tiene un descuento de $\gamma = 0,9$.

El "Programa-Agente" se puede obtener creando una instancia de la "Clase-Agente" correspondiente. El método `__call__` permite llamar a la `Clase-Agente` como una función. Es necesario crear una instancia de la clase con una política ($\pi$) y un "MDP" cuya utilidad de los estados se estimará.

In [ ]:
from mdp import sequential_decision_environment

El `sequential_decision_environment` es un objeto GridMDP como se muestra a continuación. Las recompensas son **+1** y **-1** en los estados terminales, y **-0,04** en el resto. <img src="files/images/mdp.png"> Ahora definimos acciones y una política similar a **Fig 21.1** en el libro.

In [ ]:
# Action Directions
north = (0, 1)
south = (0,-1)
west = (-1, 0)
east = (1, 0)

policy = {
    (0, 2): east,  (1, 2): east,  (2, 2): east,   (3, 2): None,
    (0, 1): north,                (2, 1): north,  (3, 1): None,
    (0, 0): north, (1, 0): west,  (2, 0): west,   (3, 0): west, 
}


### Agente de estimación de servicios públicos de dirección

La clase `PassiveDEUAgent` en el módulo `rl` implementa el Programa Agente descrito en **Fig 21.2** del Libro AIMA. `PassiveDEUAgent` suma las recompensas para encontrar la utilidad estimada para cada estado. Por tanto, requiere la ejecución de varias iteraciones.

In [ ]:
%psource PassiveDUEAgent

In [ ]:
DUEagent = PassiveDUEAgent(policy, sequential_decision_environment)
for i in range(200):
    run_single_trial(DUEagent, sequential_decision_environment)
    DUEagent.estimate_U()



Las utilidades calculadas son:

In [ ]:
print('\n'.join([str(k)+':'+str(v) for k, v in DUEagent.U.items()]))

### Agente de programación dinámica adaptativa

La clase `PassiveADPAgent` en el módulo `rl` implementa el Programa Agente descrito en **Fig 21.2** del Libro AIMA. `PassiveADPAgent` utiliza transición de estado y recuentos de ocurrencias para estimar $P$ y luego $U$. Consulte la fuente a continuación para comprender al agente.

In [ ]:
%psource PassiveADPAgent

Creamos una instancia de un `PassiveADPAgent` a continuación con el `GridMDP` que se muestra y lo entrenamos en más de 200 iteraciones. El módulo `rl` tiene una implementación simple para simular iteraciones. La función se llama **run_single_trial**.

In [ ]:
ADPagent = PassiveADPAgent(policy, sequential_decision_environment)
for i in range(200):
    run_single_trial(ADPagent, sequential_decision_environment)

Las utilidades calculadas son:

In [ ]:
print('\n'.join([str(k)+':'+str(v) for k, v in ADPagent.U.items()]))

### Agente pasivo de diferencia temporal

"PassiveTDAgent" utiliza diferencias temporales para conocer estimaciones de utilidad. Aprendemos la diferencia entre los estados y respaldamos los valores en estados anteriores. Analicemos la fuente antes de ver algunos ejemplos de uso.

In [ ]:
%psource PassiveTDAgent

Al crear el `TDAgent`, utilizamos la **misma tasa de aprendizaje** $\alpha$ que se indica en la nota al pie del libro en la **página 837**.

In [ ]:
TDagent = PassiveTDAgent(policy, sequential_decision_environment, alpha = lambda n: 60./(59+n))

Ahora realizamos **200 pruebas** para que el agente estime las utilidades.

In [ ]:
for i in range(200):
    run_single_trial(TDagent,sequential_decision_environment)

Las utilidades calculadas son:

In [ ]:
print('\n'.join([str(k)+':'+str(v) for k, v in TDagent.U.items()]))

## Comparación con el método de iteración de valores.

También podemos comparar las estimaciones de utilidad aprendidas por nuestro agente con las obtenidas mediante **iteración de valor**.

**Tenga en cuenta que la iteración de valor tiene conocimiento a priori de la tabla de transición $P$, las recompensas $R$ y todos los estados $s$.**

In [ ]:
from mdp import value_iteration

Los valores calculados por iteración de valores:

In [ ]:
U_values = value_iteration(sequential_decision_environment)
print('\n'.join([str(k)+':'+str(v) for k, v in U_values.items()]))

## Evolución de las estimaciones de utilidad a lo largo de iteraciones

Podemos explorar cómo varían estas estimaciones con el tiempo utilizando gráficos similares a **Figura 21.5a**. Primero habilitaremos matplotlib usando el backend en línea. También definimos una función para recopilar los valores de las utilidades en cada iteración.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

def graph_utility_estimates(agent_program, mdp, no_of_iterations, states_to_graph):
    graphs = {state:[] for state in states_to_graph}
    for iteration in range(1,no_of_iterations+1):
        run_single_trial(agent_program, mdp)
        for state in states_to_graph:
            graphs[state].append((iteration, agent_program.U[state]))
    for state, value in graphs.items():
        state_x, state_y = zip(*value)
        plt.plot(state_x, state_y, label=str(state))
    plt.ylim([0,1.2])
    plt.legend(loc='lower right')
    plt.xlabel('Iterations')
    plt.ylabel('U')

Aquí hay un gráfico del estado $(2,2)$.

In [ ]:
agent = PassiveTDAgent(policy, sequential_decision_environment, alpha=lambda n: 60./(59+n))
graph_utility_estimates(agent, sequential_decision_environment, 500, [(2,2)])

También es posible trazar varios estados en el mismo gráfico. Como se esperaba, la utilidad del estado finito $(3,2)$ permanece constante y es igual a $R((3,2)) = 1$.

In [ ]:
graph_utility_estimates(agent, sequential_decision_environment, 500, [(2,2), (3,2)])

##APRENDIZAJE ACTIVO POR REFUERZO

A diferencia del aprendizaje por refuerzo pasivo, en el aprendizaje por refuerzo activo no estamos sujetos a una política pi y debemos seleccionar nuestras acciones. En otras palabras, el agente necesita aprender una política óptima. El equilibrio fundamental que el agente debe afrontar es el de exploración versus explotación.

### QAgente de aprendizaje

La clase QLearningAgent en el módulo rl implementa el Programa Agente descrito en **Fig 21.8** del Libro AIMA. En Q-Learning, el agente aprende una función de valor de acción Q que le brinda la utilidad de realizar una acción determinada en un estado particular. Q-Learning no requiere un modelo de transición y, por lo tanto, es un método sin modelos. Analicemos la fuente antes de ver algunos ejemplos de uso.

In [ ]:
%psource QLearningAgent

El Programa Agente se puede obtener creando la instancia de la clase pasando los parámetros apropiados. Debido al método __ call __, el objeto que se crea se comporta como un invocable y devuelve una acción apropiada como lo hacen la mayoría de los programas de agentes. Para crear una instancia del objeto necesitamos un mdp similar al PassiveTDAgent.

Usemos el mismo objeto GridMDP que usamos anteriormente. **La Figura 17.1 (entorno_de decisión_sequential)** es similar a la **Figura 21.1** pero tiene algunos descuentos como **gamma = 0,9**. La clase también implementa una función de exploración **f** que devuelve **Rplus** fijo hasta que el agente haya visitado el estado, acción **Ne** número de veces. Este es el mismo que se define en la página **842** del libro. El método **actions_in_state** devuelve acciones posibles en un estado determinado. Es útil al aplicar operaciones max y argmax.

Creemos nuestro objeto ahora. También utilizamos el **mismo alfa** que figura en la nota al pie del libro en la **página 837**. Usamos **Rplus = 2** y **Ne = 5** como se define en la página 843. **Fig 21.7**

In [ ]:
q_agent = QLearningAgent(sequential_decision_environment, Ne=5, Rplus=2, 
                         alpha=lambda n: 60./(59+n))

Ahora, para probar q_agent utilizamos la función **run_single_trial** en rl.py (que también se usó anteriormente). Usemos **200** iteraciones.

In [ ]:
for i in range(200):
    run_single_trial(q_agent,sequential_decision_environment)

Ahora veamos los valores Q. Las claves son pares estado-acción. Donde corresponden diferentes acciones según:

norte = (0, 1)
sur = (0,-1)
oeste = (-1, 0)
este = (1, 0)

In [ ]:
q_agent.Q

La Utilidad **U** de cada estado está relacionada con **Q** mediante la siguiente ecuación.

**U(s) = máx <sub>a</sub> Q(s, a)**

Convirtamos los valores Q anteriores en estimaciones U.



In [ ]:
U = defaultdict(lambda: -1000.) # Very Large Negative Value for Comparison see below.
for state_action, value in q_agent.Q.items():
    state, action = state_action
    if U[state] < value:
                U[state] = value

In [ ]:
U

Finalmente comparemos estas estimaciones con los resultados de value_iteration.

In [ ]:
print(value_iteration(sequential_decision_environment))